In [ ]:
import os
os._exit(00)


In [ ]:
!pip install transformers datasets torch

# Step 2: Import necessary libraries
import os
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

In [ ]:
def load_ner_dataset(folder_path):
    sentences = []
    labels = []
    for file_name in os.listdir(folder_path):
        with open(os.path.join(folder_path, file_name), 'r', encoding='ISO-8859-1') as file:  # Specify encoding
            tokens = []
            ner_tags = []
            for line in file:
                if line.strip():
                    parts = line.strip().split()
                    token = " ".join(parts[:-1])
                    label = parts[-1]
                    tokens.append(token)
                    ner_tags.append(label)
                else:
                    sentences.append(tokens)
                    labels.append(ner_tags)
                    tokens = []
                    ner_tags = []
            if tokens:
                sentences.append(tokens)
                labels.append(ner_tags)
    return sentences, labels

In [ ]:
folder_path = '/content/annotated'
sentences, labels = load_ner_dataset(folder_path)

In [ ]:
print(len(sentences ))
print(len(labels ))
print(sentences)
print(labels)

204
204
[['Medical', 'Record'], ['Patient', 'Information:'], ['Name:', 'John', 'Doe'], ['Date', 'of', 'Birth:', 'January', '15,', '1985'], ['Gender:', 'Male'], ['Patient', 'ID:', '123456'], ['Date', 'of', 'Visit:', 'March', '8,', '2000'], ['Chief', 'Complaint:'], ['Pain', 'in', 'the', 'neck', 'and', 'upper', 'back'], ['History', 'of', 'Present', 'Illness', '(HPI):'], ['Onset:', 'The', 'pain', 'started', 'approximately', '1', 'month', 'ago.'], ['Location:', 'Cervical', 'region', '(neck', 'and', 'upper', 'back).'], ['Duration:', 'Persistent,', 'worsening', 'over', 'time.'], ['Characteristics:', 'Dull,', 'aching', 'pain', 'with', 'occasional', 'sharp,', 'shooting', 'pains', 'down', 'the', 'arms.'], ['Aggravating', 'Factors:', 'Prolonged', 'sitting,', 'certain', 'neck', 'movements.'], ['Relieving', 'Factors:', 'Rest,', 'over-the-counter', 'pain', 'medications,', 'gentle', 'neck', 'exercises.'], ['Associated', 'Symptoms:', 'Numbness', 'and', 'tingling', 'in', 'the', 'arms,', 'occasional', '

In [ ]:
sentences = [item for sublist in sentences for item in sublist]
labels = [item for sublist in labels for item in sublist]

In [ ]:
print(len(sentences ))
print(len(labels ))
print(sentences)
print(labels)

1268
1268
['Medical', 'Record', 'Patient', 'Information:', 'Name:', 'John', 'Doe', 'Date', 'of', 'Birth:', 'January', '15,', '1985', 'Gender:', 'Male', 'Patient', 'ID:', '123456', 'Date', 'of', 'Visit:', 'March', '8,', '2000', 'Chief', 'Complaint:', 'Pain', 'in', 'the', 'neck', 'and', 'upper', 'back', 'History', 'of', 'Present', 'Illness', '(HPI):', 'Onset:', 'The', 'pain', 'started', 'approximately', '1', 'month', 'ago.', 'Location:', 'Cervical', 'region', '(neck', 'and', 'upper', 'back).', 'Duration:', 'Persistent,', 'worsening', 'over', 'time.', 'Characteristics:', 'Dull,', 'aching', 'pain', 'with', 'occasional', 'sharp,', 'shooting', 'pains', 'down', 'the', 'arms.', 'Aggravating', 'Factors:', 'Prolonged', 'sitting,', 'certain', 'neck', 'movements.', 'Relieving', 'Factors:', 'Rest,', 'over-the-counter', 'pain', 'medications,', 'gentle', 'neck', 'exercises.', 'Associated', 'Symptoms:', 'Numbness', 'and', 'tingling', 'in', 'the', 'arms,', 'occasional', 'headaches.', 'Past', 'Medical',

In [ ]:
unique_labels = set(labels)
tag2idx = {tag: idx for idx, tag in enumerate(unique_labels)}
tag2idx["PAD"] = len(tag2idx)

In [ ]:
if "mmHg." in tag2idx:
    del tag2idx["mmHg."]


In [ ]:

print(tag2idx)

{'B-VISITDATE': 0, 'I-DIAGNOSE': 1, 'mmHg.': 2, 'I-VISITDATE': 3, 'B-DIAGNOSE': 4, 'O': 5, 'I-DIADNOSE': 6, 'PAD': 7}


In [ ]:
    model_name = "dmis-lab/biobert-base-cased-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize_and_preserve_labels(sentences, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentences, text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels


In [ ]:
tokenized_sentence, labels = tokenize_and_preserve_labels(sentences, labels)

In [ ]:
print(len(tokenized_sentence))
print(len(labels))
print(tokenized_sentence)
print(labels)

2227
2227
['medical', 'record', 'patient', 'information', ':', 'name', ':', 'j', '##oh', '##n', 'do', '##e', 'date', 'of', 'birth', ':', 'j', '##anu', '##ary', '15', ',', '1985', 'gender', ':', 'male', 'patient', 'id', ':', '123', '##45', '##6', 'date', 'of', 'visit', ':', 'march', '8', ',', '2000', 'chief', 'complaint', ':', 'pain', 'in', 'the', 'neck', 'and', 'upper', 'back', 'history', 'of', 'present', 'illness', '(', 'hp', '##i', ')', ':', 'onset', ':', 'the', 'pain', 'started', 'approximately', '1', 'month', 'ago', '.', 'location', ':', 'c', '##er', '##vic', '##al', 'region', '(', 'neck', 'and', 'upper', 'back', ')', '.', 'duration', ':', 'persistent', ',', 'worse', '##ning', 'over', 'time', '.', 'characteristics', ':', 'dull', ',', 'aching', 'pain', 'with', 'occasional', 'sharp', ',', 'shooting', 'pains', 'down', 'the', 'arms', '.', 'a', '##gg', '##ra', '##vat', '##ing', 'factors', ':', 'prolonged', 'sitting', ',', 'certain', 'neck', 'movements', '.', 're', '##lie', '##ving', 'fa

In [ ]:
input_ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)
label_ids = [tag2idx[label] for label in labels]


In [ ]:
print(len(input_ids))
print(len(label_ids))
print(input_ids)
print(label_ids)

2227
2227
[2657, 1647, 5351, 1869, 131, 1271, 131, 179, 10559, 1179, 1202, 1162, 2236, 1104, 3485, 131, 179, 19762, 3113, 1405, 117, 2210, 5772, 131, 2581, 5351, 25021, 131, 13414, 21336, 1545, 2236, 1104, 3143, 131, 8943, 129, 117, 1539, 2705, 12522, 131, 2489, 1107, 1103, 2455, 1105, 3105, 1171, 1607, 1104, 1675, 6946, 113, 6857, 1182, 114, 131, 15415, 131, 1103, 2489, 1408, 2324, 122, 2370, 2403, 119, 2450, 131, 172, 1200, 15901, 1348, 1805, 113, 2455, 1105, 3105, 1171, 114, 119, 9355, 131, 15970, 117, 4146, 3381, 1166, 1159, 119, 5924, 131, 10884, 117, 15790, 2489, 1114, 7957, 4295, 117, 4598, 24782, 1205, 1103, 1739, 119, 170, 9705, 1611, 26465, 1158, 5320, 131, 16639, 2807, 117, 2218, 2455, 5172, 119, 1231, 7174, 3970, 5320, 131, 1832, 117, 1166, 118, 1103, 118, 4073, 2489, 23897, 117, 6892, 2455, 11536, 119, 2628, 8006, 131, 18152, 1757, 1105, 26882, 1107, 1103, 1739, 117, 7957, 16320, 1116, 119, 1763, 2657, 1607, 131, 13306, 2975, 131, 177, 24312, 23826, 1988, 117, 2076, 123, 1

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_LEN = 512
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(tokenized_sentence)],
                          maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

label_ids = pad_sequences([[tag2idx.get(l) for l in labels]],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post", dtype="long", truncating="post")

In [ ]:
print(len(input_ids))
print(len(label_ids))
print(input_ids)
print(label_ids)

1
1
[[ 2657  1647  5351  1869   131  1271   131   179 10559  1179  1202  1162
   2236  1104  3485   131   179 19762  3113  1405   117  2210  5772   131
   2581  5351 25021   131 13414 21336  1545  2236  1104  3143   131  8943
    129   117  1539  2705 12522   131  2489  1107  1103  2455  1105  3105
   1171  1607  1104  1675  6946   113  6857  1182   114   131 15415   131
   1103  2489  1408  2324   122  2370  2403   119  2450   131   172  1200
  15901  1348  1805   113  2455  1105  3105  1171   114   119  9355   131
  15970   117  4146  3381  1166  1159   119  5924   131 10884   117 15790
   2489  1114  7957  4295   117  4598 24782  1205  1103  1739   119   170
   9705  1611 26465  1158  5320   131 16639  2807   117  2218  2455  5172
    119  1231  7174  3970  5320   131  1832   117  1166   118  1103   118
   4073  2489 23897   117  6892  2455 11536   119  2628  8006   131 18152
   1757  1105 26882  1107  1103  1739   117  7957 16320  1116   119  1763
   2657  1607   131 13306  2975   

In [ ]:
from torch.utils.data import Dataset

In [ ]:

class NERDataset(Dataset):
    def __init__(self, input_ids, label_ids):
        self.input_ids = input_ids
        self.label_ids = label_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'labels': self.label_ids[idx]
        }

In [ ]:
from torch.utils.data import DataLoader

# Create the dataset
dataset = NERDataset(input_ids, label_ids)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)




In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3000,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="no",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(tag2idx))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def save_model(self, output_dir=None, _internal_call=False):
        # Ensure tensors are contiguous before saving
        for param in self.model.parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()

        # Now save the model
        super().save_model(output_dir, _internal_call=_internal_call)

# Use the custom trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Train the model
trainer.train()


Step,Training Loss
10,0.354100
20,0.040500
30,0.013000
40,0.007800
50,0.006000
60,0.004300
70,0.004700
80,0.001800
90,0.000700
100,0.000400


TrainOutput(global_step=3000, training_loss=0.0014913750778214308, metrics={'train_runtime': 520.9676, 'train_samples_per_second': 5.759, 'train_steps_per_second': 5.759, 'total_flos': 783932792832000.0, 'train_loss': 0.0014913750778214308, 'epoch': 3000.0})

In [ ]:
# Ensure tensors are contiguous before saving
for param in model.parameters():
    param.data = param.data.contiguous()

# Save the model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

#Yeah Its Time for  Testing

In [ ]:
import nltk

# Download the punkt tokenizer models
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from nltk import sent_tokenize

# Download the punkt tokenizer models (if not already downloaded)
import nltk
nltk.download('punkt')

# Load the model and tokenizer
model_path = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

# Input text for NER
text = """Patient Information:
Medical Record
Name: John Doe
Date of Birth: January 15, 1985
Gender: Male
Patient ID: 123456
Visited: 20/08/2020
Chief Complaint:
Pain in the right ankle and joint of the right foot
History of Present Illness (HPI):
Onset: The pain started approximately 3 weeks ago after a minor sprain.
Location: Right ankle and joint of the right foot.
Duration: Persistent, worsening with activity.
Characteristics: Sharp, throbbing pain with swelling and tenderness.
Aggravating Factors: Walking, standing for long periods, and physical activity.
Relieving Factors: Rest, ice application, elevation, and over-the-counter pain medications (ibuprofen). Associated Symptoms: Swelling, stiffness, and occasional bruising.
Past Medical History:
Chronic Conditions: Hypertension, Type 2 Diabetes.
Previous Surgeries: Appendectomy (2010).
Medications: Metformin, Lisinopril, Ibuprofen (as needed for pain).
Allergies: No known drug allergies.
Social History:
Occupation: Office worker.
Smoking: Non-smoker.
Alcohol Use: Occasionally.

Exercise: Infrequent.
Family History:
Father: Hypertension, Heart disease.
Mother: Type 2 Diabetes.
Siblings: Healthy.
Review of Systems:
General: No fever, no weight loss.
Musculoskeletal: Pain in the right ankle and joint of the right foot, no other joint pain or swelling.
Neurological: No numbness or tingling.
Cardiovascular: No chest pain, no palpitations.
Respiratory: No shortness of breath, no cough.
Physical Examination:
General: Alert, well-nourished, in no acute distress.
Vital Signs:
Blood Pressure: 130/85 mmHg.
Heart Rate: 78 bpm.
Respiratory Rate: 16 breaths/min.
Temperature: 98.6°F (37°C).
Inspection: Swelling and mild bruising around the right ankle and foot.
Palpation: Tenderness over the lateral aspect of the right ankle and dorsum of the right foot.
Range of Motion: Limited due to pain and swelling, especially in dorsiflexion and plantarflexion.
Strength: 4/5 in the right lower extremity.
Reflexes: Normal deep tendon reflexes in the lower extremities.
Sensation: Intact sensation in the right foot and ankle.
Special Tests: Negative anterior drawer test, negative talar tilt test indicating no major ligamentous instability.
Assessment:

Prognosis: Right ankle sprain with possible involvement of the joint of the right foot. Differential Prognosis: Ankle fracture, tendonitis, arthritis.
Plan:
Imaging: Order X-ray of the right ankle and foot to rule out fracture.
Medications: Continue ibuprofen for pain management; consider prescribing a stronger anti-inflammatory if necessary.
Referral: Refer to orthopedic specialist for further evaluation if no improvement.
Physical Therapy: Recommend physical therapy for rehabilitation and strengthening exercises.
Activity: Advise patient to avoid weight-bearing activities on the affected foot; use crutches if necessary.
Follow-Up: Schedule follow-up appointment in 2 weeks to review X-ray results and assess response to treatment.
Patient Education:
Discussed the nature of the condition, potential causes, and treatment options.
Instructed on proper use of medication and importance of adhering to prescribed therapy.
Encouraged patient to report any worsening symptoms or new concerns.

"""

# Split text into sentences to process each sentence separately
sentences = sent_tokenize(text)

# Prepare list to hold final results
all_predictions = []

# Iterate over sentences and perform NER
for sentence in sentences:
    # Tokenize the sentence with truncation
    inputs = tokenizer(sentence.split(), return_tensors="pt", is_split_into_words=True, truncation=True, max_length=512)

    # Get predictions from the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class indices
    predictions = torch.argmax(outputs.logits, dim=2)

    # Reverse the tag2idx dictionary to get idx2tag
    idx2tag = {idx: tag for tag, idx in tag2idx.items()}

    # Convert predicted indices to actual entity labels
    predicted_labels = [idx2tag[pred.item()] for pred in predictions[0]]

    # Get the tokens from the tokenizer
    tokens_chunk = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Collect the results, filtering out special tokens
    all_predictions.extend([(token, label) for token, label in zip(tokens_chunk, predicted_labels) if token not in ["[PAD]", "[CLS]", "[SEP]"]])

# Compile the results into the full text with labels
compiled_text = []
for token, label in all_predictions:
    compiled_text.append(f"{token}({label})")

# Join the compiled text to display as a full sentence
compiled_text = " ".join(compiled_text)
print(compiled_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


patient(O) information(O) :(O) medical(O) record(O) name(O) :(O) j(O) ##oh(O) ##n(O) do(O) ##e(O) date(O) of(O) birth(O) :(O) j(O) ##anu(O) ##ary(O) 15(O) ,(O) 1985(O) gender(O) :(O) male(O) patient(O) id(O) :(O) 123(O) ##45(O) ##6(O) visited(O) :(O) 20(I-VISITDATE) /(I-VISITDATE) 08(I-VISITDATE) /(I-VISITDATE) 2020(I-VISITDATE) chief(O) complaint(O) :(O) pain(O) in(O) the(O) right(O) ankle(O) and(O) joint(O) of(O) the(O) right(O) foot(O) history(O) of(O) present(O) illness(O) ((O) hp(O) ##i(O) )(O) :(O) onset(O) :(O) the(O) pain(O) started(O) approximately(O) 3(O) weeks(O) ago(O) after(O) a(O) minor(O) s(O) ##pra(O) ##in(O) .(O) location(O) :(O) right(O) ankle(O) and(O) joint(O) of(O) the(O) right(O) foot(O) .(O) duration(O) :(O) persistent(O) ,(O) worse(O) ##ning(O) with(O) activity(O) .(O) characteristics(O) :(O) sharp(O) ,(O) throbbing(O) pain(O) with(O) swelling(O) and(O) tender(O) ##ness(O) .(O) a(O) ##gg(O) ##ra(O) ##vat(O) ##ing(O) factors(O) :(O) walking(O) ,(O) standing(O) fo

In [1]:
import re

# The text input with token-label pairs
input_text = '''patient(O) information(O) :(O) medical(O) record(O) name(O) :(O) j(O) ##oh(O) ##n(O) do(O) ##e(O) date(O) of(O) birth(O) :(O) j(O) ##anu(O) ##ary(O) 15(O) ,(O) 1985(O) gender(O) :(O) male(O) patient(O) id(O) :(O) 123(O) ##45(O) ##6(O) visited(O) :(O) 20(I-VISITDATE) /(I-VISITDATE) 08(I-VISITDATE) /(I-VISITDATE) 2020(I-VISITDATE) chief(O) complaint(O) :(O) pain(O) in(O) the(O) right(O) ankle(O) and(O) joint(O) of(O) the(O) right(O) foot(O) history(O) of(O) present(O) illness(O) ((O) hp(O) ##i(O) )(O) :(O) onset(O) :(O) the(O) pain(O) started(O) approximately(O) 3(O) weeks(O) ago(O) after(O) a(O) minor(O) s(O) ##pra(O) ##in(O) .(O) location(O) :(O) right(O) ankle(O) and(O) joint(O) of(O) the(O) right(O) foot(O) .(O) duration(O) :(O) persistent(O) ,(O) worse(O) ##ning(O) with(O) activity(O) .(O) characteristics(O) :(O) sharp(O) ,(O) throbbing(O) pain(O) with(O) swelling(O) and(O) tender(O) ##ness(O) .(O) a(O) ##gg(O) ##ra(O) ##vat(O) ##ing(O) factors(O) :(O) walking(O) ,(O) standing(O) for(O) long(O) periods(O) ,(O) and(O) physical(O) activity(O) .(O) re(O) ##lie(O) ##ving(O) factors(O) :(O) rest(O) ,(O) ice(O) application(O) ,(O) elevation(O) ,(O) and(O) over(O) -(O) the(O) -(O) counter(O) pain(O) medications(O) ((O) i(O) ##bu(O) ##p(O) ##ro(O) ##fen(O) )(O) .(O) associated(O) symptoms(O) :(O) swelling(O) ,(O) stiff(O) ##ness(O) ,(O) and(O) occasional(O) br(O) ##ui(O) ##sing(O) .(O) past(O) medical(O) history(O) :(O) chronic(O) conditions(O) :(O) h(O) ##yper(O) ##tens(O) ##ion(O) ,(O) type(O) 2(O) diabetes(O) .(O) previous(O) surge(O) ##ries(O) :(O) app(O) ##end(O) ##ec(O) ##tom(O) ##y(O) ((O) 2010(O) )(O) .(O) medications(O) :(O) met(O) ##form(O) ##in(O) ,(O) l(O) ##isi(O) ##no(O) ##p(O) ##ril(O) ,(O) i(O) ##bu(O) ##p(O) ##ro(O) ##fen(O) ((O) as(O) needed(O) for(O) pain(O) )(O) .(O) all(O) ##er(O) ##gies(O) :(O) no(O) known(O) drug(O) all(O) ##er(O) ##gies(O) .(O) social(O) history(O) :(O) occupation(O) :(O) office(O) worker(O) .(O) smoking(O) :(O) non(O) -(O) smoke(O) ##r(O) .(O) alcohol(O) use(O) :(O) occasionally(O) .(O) exercise(O) :(O) in(O) ##f(O) ##re(O) ##quent(O) .(O) family(O) history(O) :(O) father(O) :(O) h(O) ##yper(O) ##tens(O) ##ion(O) ,(O) heart(O) disease(O) .(O) mother(O) :(O) type(O) 2(O) diabetes(O) .(O) siblings(O) :(O) healthy(O) .(O) review(O) of(O) systems(O) :(O) general(O) :(O) no(O) fever(O) ,(O) no(O) weight(O) loss(O) .(O) m(O) ##us(O) ##cu(O) ##los(O) ##kel(O) ##etal(O) :(O) pain(O) in(O) the(O) right(O) ankle(O) and(O) joint(O) of(O) the(O) right(O) foot(O) ,(O) no(O) other(O) joint(O) pain(O) or(O) swelling(O) .(O) ne(O) ##uro(O) ##logical(O) :(O) no(O) numb(O) ##ness(O) or(O) tingling(O) .(O) card(O) ##iovascular(O) :(O) no(O) chest(O) pain(O) ,(O) no(O) p(O) ##al(O) ##pit(O) ##ations(O) .(O) respiratory(O) :(O) no(O) short(O) ##ness(O) of(O) breath(O) ,(O) no(O) cough(O) .(O) physical(O) examination(O) :(O) general(O) :(O) alert(O) ,(O) well(O) -(O) no(O) ##uri(O) ##shed(O) ,(O) in(O) no(O) acute(O) distress(O) .(O) vital(O) signs(O) :(O) blood(O) pressure(O) :(O) 130(O) /(O) 85(O) mm(O) ##h(O) ##g(O) .(O) heart(O) rate(O) :(O) 78(O) b(O) ##pm(O) .(O) respiratory(O) rate(O) :(O) 16(O) breaths(O) /(O) min(O) .(O) temperature(O) :(O) 98(O) .(O) 6(O) ##°(O) ##f(O) ((O) 37(O) ##°(O) ##c(O) )(O) .(O) inspection(O) :(O) swelling(O) and(O) mild(O) br(O) ##ui(O) ##sing(O) around(O) the(O) right(O) ankle(O) and(O) foot(O) .(O) p(O) ##al(O) ##pation(O) :(O) tender(O) ##ness(O) over(O) the(O) lateral(O) aspect(O) of(O) the(O) right(O) ankle(O) and(O) do(O) ##rs(O) ##um(O) of(O) the(O) right(O) foot(O) .(O) range(O) of(O) motion(O) :(O) limited(O) due(O) to(O) pain(O) and(O) swelling(O) ,(O) especially(O) in(O) do(O) ##rs(O) ##if(O) ##lex(O) ##ion(O) and(O) plant(O) ##ar(O) ##fle(O) ##xi(O) ##on(O) .(O) strength(O) :(O) 4(O) /(O) 5(O) in(O) the(O) right(O) lower(O) ex(O) ##tre(O) ##mity(O) .(O) reflex(O) ##es(O) :(O) normal(O) deep(O) tend(O) ##on(O) reflex(O) ##es(O) in(O) the(O) lower(O) ex(O) ##tre(O) ##mit(O) ##ies(O) .(O) sensation(O) :(O) intact(O) sensation(O) in(O) the(O) right(O) foot(O) and(O) ankle(O) .(O) special(O) tests(O) :(O) negative(O) anterior(O) drawer(O) test(O) ,(O) negative(O) ta(O) ##lar(O) tilt(O) test(O) indicating(O) no(O) major(O) l(O) ##iga(O) ##mento(O) ##us(O) instability(O) .(O) assessment(O) :(O) pro(O) ##gno(O) ##sis(O) :(O) right(O) ankle(O) s(O) ##pra(O) ##in(O) with(O) possible(O) involvement(O) of(O) the(O) joint(O) of(O) the(O) right(O) foot(O) .(O) differential(O) pro(O) ##gno(O) ##sis(O) :(O) ankle(O) fracture(O) ,(O) tend(O) ##oni(O) ##tis(O) ,(O) art(O) ##hr(O) ##itis(O) .(O) plan(O) :(O) imaging(O) :(O) order(O) x(O) -(O) ray(O) of(O) the(O) right(O) ankle(O) and(O) foot(O) to(O) rule(O) out(O) fracture(O) .(O) medications(O) :(O) continue(O) i(O) ##bu(O) ##p(O) ##ro(O) ##fen(O) for(O) pain(O) management(O) ;(O) consider(O) pre(O) ##s(O) ##cribing(O) a(O) stronger(O) anti(O) -(O) inflammatory(O) if(O) necessary(O) .(O) refer(O) ##ral(O) :(O) refer(O) to(O) or(O) ##th(O) ##oped(O) ##ic(O) specialist(O) for(O) further(O) evaluation(O) if(O) no(O) improvement(O) .(O) physical(O) therapy(O) :(O) recommend(O) physical(O) therapy(O) for(O) rehabilitation(O) and(O) strengthening(O) exercises(O) .(O) activity(O) :(O) advise(O) patient(O) to(O) avoid(O) weight(O) -(O) bearing(O) activities(O) on(O) the(O) affected(O) foot(O) ;(O) use(O) c(O) ##ru(O) ##tches(O) if(O) necessary(O) .(O) follow(O) -(O) up(O) :(O) schedule(O) follow(O) -(O) up(O) appointment(O) in(O) 2(O) weeks(O) to(O) review(O) x(O) -(O) ray(O) results(O) and(O) assess(O) response(O) to(O) treatment(O) .(O) patient(O) education(O) :(O) discussed(O) the(O) nature(O) of(O) the(O) condition(O) ,(O) potential(O) causes(O) ,(O) and(O) treatment(O) options(O) .(O) instructed(O) on(O) proper(O) use(O) of(O) medication(O) and(O) importance(O) of(O) ad(O) ##hering(O) to(O) prescribed(O) therapy(O) .(O) encouraged(O) patient(O) to(O) report(O) any(O) worse(O) ##ning(O) symptoms(O) or(O) new(O) concerns(O) .(O)'''

# Function to process text and join entities
def join_entities(text):
    # Split the text into token-label pairs
    token_label_pairs = re.findall(r'(\S+)\((\S+)\)', text)

    # Initialize variables
    result = []
    current_entity = ""
    current_label = ""

    # Process token-label pairs
    for token, label in token_label_pairs:
        if label == 'O':
            continue  # Skip 'O' labeled tokens

        if label == current_label:
            current_entity += token.replace('##', '')  # Continue joining the token
        else:
            if current_entity:
                result.append(f"{current_entity} ({current_label})")  # Append previous entity
            current_entity = token.replace('##', '')  # Start new entity
            current_label = label

    # Append the last entity
    if current_entity:
        result.append(f"{current_entity} ({current_label})")

    return result

# Call the function and print results
entities = join_entities(input_text)
for entity in entities:
    print(entity)


20/08/2020 (I-VISITDATE)
